<a href="https://colab.research.google.com/github/lucifer6660/Credit-Card-Fraudlent/blob/master/NLP_Haclathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the dataset and libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets evaluate



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.2 MB/s eta 0:00:00


In [3]:
from datasets import Dataset
import numpy as np

In [4]:
# Data processing
import pandas as pd
import numpy as np

# Modeling
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TextClassificationPipeline

In [5]:
# Model performance evaluation
import evaluate

In [7]:
df=pd.read_csv('/content/drive/MyDrive/new/Copy of PS3_train.csv')

In [8]:
df.head()

,content,title,uid,target_ind
0,Premium quality five pocket jean from Wrangler...,Amazon.com: Wrangler Men's Rugged Wear Relaxed...,B0000CBALT,247
1,If you're looking for a different kind of anim...,Sakura Diaries - Complete Series Collector's E...,B00005QFDT,453
2,"First things first: Yes, Thinking XXX features...",Thinking XXX (Extended Cut) (2006),B000BNXD50,228
3,Feathertouch. 100% Polyester Machine Wash Warm...,Amazon.com: Petite Feathertouch Pull-On Pant: ...,B0002LK9V2,223
4,"When you need outstanding fuel delivery, easy ...",ACDelco EP386 Fuel Pump,B000C9PA54,312


In [9]:
len(df)

35112

In [10]:
df.describe()

,target_ind
count,35112.000000
mean,263.933157
std,135.519242
min,0.000000
25%,141.000000
50%,288.000000
75%,361.000000
max,499.000000


In [11]:
df['target_ind'].value_counts()

348    1243
351    1212
349     622
298     445
91      439
       ... 
14       35
137      35
377      35
19       35
170      35
Name: target_ind, Length: 500, dtype: int64

In [12]:
len(set(df['target_ind']))

500

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35112 entries, 0 to 35111
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   content     35112 non-null  object
 1   title       35112 non-null  object
 2   uid         35112 non-null  object
 3   target_ind  35112 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.1+ MB


In [14]:
df.isnull()

,content,title,uid,target_ind
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
35107,False,False,False,False
35108,False,False,False,False
35109,False,False,False,False
35110,False,False,False,False


In [15]:
df.isna().any()

content       False
title         False
uid           False
target_ind    False
dtype: bool

## Merging 2 columns and dropping id

In [16]:
# Using apply() method to combine two columns of text
df["review"] = df[["content", "title"]].apply("-".join, axis=1)

In [17]:
df.drop('uid',axis=1,inplace=True)

In [18]:
df.drop('title',axis=1,inplace=True)

In [19]:
df.drop('content',axis=1,inplace=True)

In [20]:
df.columns=['label','review']

## Splitting into train and test

In [21]:
# Training dataset
train_data = df.sample(frac=0.8, random_state=42)

In [22]:
# Testing dataset
test_data = df.drop(train_data.index)

In [23]:
# Check the number of records in training and testing dataset.
print(f'The training dataset has {len(train_data)} records.')
print(f'The testing dataset has {len(test_data)} records.')

The training dataset has 28090 records.
The testing dataset has 7022 records.


In [24]:
hg_train_data = Dataset.from_pandas(train_data)
hg_test_data = Dataset.from_pandas(test_data)

## Tokenizer

In [25]:
# Tokenizer from a pretrained model
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [26]:
# Take a look at the tokenizer
tokenizer

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [27]:
# Funtion to tokenize data
def tokenize_dataset(data):
    return tokenizer(data["review"], 
                     max_length=32, 
                     truncation=True, 
                     padding="max_length")

# Tokenize the dataset
dataset_train = hg_train_data.map(tokenize_dataset)
dataset_test = hg_test_data.map(tokenize_dataset)

Map:   0%|          | 0/28090 [00:00<?, ? examples/s]

Map:   0%|          | 0/7022 [00:00<?, ? examples/s]

## Model

In [28]:
# Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=500)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

## Training

In [29]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./sentiment_transfer_learning_transformer/",          
    logging_dir='./sentiment_transfer_learning_transformer/logs',            
    logging_strategy='epoch',
    logging_steps=100,    
    num_train_epochs=5,
    per_device_train_batch_size=30,  
    per_device_eval_batch_size=30,  
    learning_rate=5e-6,
    seed=42,
    save_strategy='epoch',
    save_steps=100,
    evaluation_strategy='epoch',
    eval_steps=100,
    load_best_model_at_end=True)


In [30]:
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    # probabilities = tf.nn.softmax(logits)
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [31]:
# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)
trainer.train()

# trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: review, __index_level_0__. If review, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 28090
  Num Epochs = 5
  Instantaneous batch size per device = 30
  Total train batch size (w. parallel, distributed & accumulation) = 30
  Gradient Accumulation steps = 1
  Total optimization steps = 4685
  Number of trainable parameters = 108694772


Epoch,Training Loss,Validation Loss,Accuracy
1,5.652800,5.145100,0.146824
2,4.996800,4.692831,0.187981
3,4.649000,4.430514,0.203361
4,4.446200,4.293056,0.219738
5,4.351700,4.249323,0.220450


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: review, __index_level_0__. If review, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7022
  Batch size = 30


Saving model checkpoint to ./sentiment_transfer_learning_transformer/checkpoint-937
Configuration saved in ./sentiment_transfer_learning_transformer/checkpoint-937/config.json
Model weights saved in ./sentiment_transfer_learning_transformer/checkpoint-937/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: review, __index_level_0__. If review, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7022
  Batch size = 30
Saving model checkpoint to ./sentiment_transfer_learning_transformer/checkpoint-1874
Configuration saved in ./sentiment_transfer_learning_transformer/checkpoint-1874/config.json
Model weights saved in ./sentiment_transfer_learning_transformer/checkpoint-1874/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argumen

TrainOutput(global_step=4685, training_loss=4.819325411052561, metrics={'train_runtime': 1104.349, 'train_samples_per_second': 127.179, 'train_steps_per_second': 4.242, 'total_flos': 2319948839308800.0, 'train_loss': 4.819325411052561, 'epoch': 5.0})

# Test Data


In [ ]:
df1=pd.read_csv('/content/drive/MyDrive/Aval/Copy of PS3_test.csv')

In [ ]:
df1.shape

(8106, 3)

In [ ]:
df1.head()

,content,title,uid
0,Socket Cap Screws are reliable and durable and...,"Alloy Steel Socket Cap Screw, Flat Head, Hex S...",0
1,"This O-ring is made of black Buna-nitrile, is ...","-161 Buna O-Ring, 70A Durometer, Black, 5-1/2&...",1
2,"This Viton O-ring is black in color, round in ...","-445 Viton O-Ring, 75A Durometer, Black, 8&#03...",2
3,This 5-ounce lightweight pique knit sport shir...,Amazon.com: Port Authority Silk Touch Sport Sh...,3
4,"This O-ring is made of black Buna-nitrile, is ...","-359 Buna O-Ring, 70A Durometer, Black, 5-3/4&...",4


In [ ]:
# Using apply() method to combine two columns of text
df1["review"] = df1[["content", "title"]].apply("-".join, axis=1)

In [ ]:
df1.drop('uid',axis=1,inplace=True)

In [ ]:
df1.drop('content',axis=1,inplace=True)

In [ ]:
df1.drop('title',axis=1,inplace=True)

In [ ]:
df1


,review
0,Socket Cap Screws are reliable and durable and...
1,"This O-ring is made of black Buna-nitrile, is ..."
2,"This Viton O-ring is black in color, round in ..."
3,This 5-ounce lightweight pique knit sport shir...
4,"This O-ring is made of black Buna-nitrile, is ..."
...,...
8101,"Gerald Mohr, Cathy O'Donnell. To help her get ..."
8102,The first Go-Kart Racing game to come out for ...
8103,For almost two decades--the '60s and the '70s-...
8104,Patrick Murray's Reflections on Commercial Lif...


In [ ]:
hg_new_data = Dataset.from_pandas(df1)

In [ ]:
hg_dataset_new = hg_new_data.map(tokenize_dataset)

Map:   0%|          | 0/8106 [00:00<?, ? examples/s]

In [ ]:
import torch

checkpoint= torch.load('/content/drive/MyDrive/24hr/my_model.pth')

# Extract the model from the checkpoint dictionary
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=500)
model.load_state_dict(checkpoint)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABE

<All keys matched successfully>

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

In [ ]:
# Predictions
y_test_predict = trainer.predict(hg_dataset_new)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: review. If review are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 8106
  Batch size = 10


In [ ]:
y_test_predict

PredictionOutput(predictions=array([[-0.702918  , -0.32358682, -0.37860867, ..., -0.12810668,
        -0.59621507,  0.11256737],
       [ 0.02878677, -0.34924853, -1.5094107 , ..., -0.3909893 ,
         0.03802276,  0.51097775],
       [ 0.00948891, -0.9638396 , -1.0199895 , ..., -0.12539755,
        -0.33143413, -0.0345207 ],
       ...,
       [-0.3224829 , -0.46531904, -0.98982465, ...,  0.3680759 ,
         1.8499101 ,  0.3367676 ],
       [ 1.7566644 ,  2.7153087 ,  1.901188  , ...,  0.11210324,
         0.51745963,  0.38554662],
       [ 0.4240297 ,  0.45465672, -0.634924  , ..., -0.9828181 ,
         1.2058283 ,  1.0455121 ]], dtype=float32), label_ids=None, metrics={'test_runtime': 17.2148, 'test_samples_per_second': 470.873, 'test_steps_per_second': 47.11})

In [ ]:
target_ind=np.argmax(y_test_predict.predictions,axis=1)

In [ ]:
target_ind

array([361, 351, 349, ...,  98,   0, 453])

In [ ]:
df1=pd.read_csv('/content/drive/MyDrive/Aval/Copy of PS3_test.csv')
df1.drop(["content","title"],inplace=True,axis=1)
df1["target_ind"]=target_ind
df1

,uid,target_ind
0,0,361
1,1,351
2,2,349
3,3,246
4,4,351
...,...,...
8101,8101,498
8102,8102,72
8103,8103,492
8104,8104,108


In [ ]:
df1.to_csv("submission2.csv",index=False)